In [ ]:
import pandas as pd
import numpy  as np
from math import sin, cos, pi, sqrt
import random
import time
import sys
sys.path.insert(1, '../framework')
from racetrack import *
rt = RACETrack()
# Test out the closest point of segment
my_pts      = [(100,200),(50,150),(250,200),(50,10)]
my_segments = [((25,5),(5,290)),((45,45),(200,220))]
svg6 =  '<svg x="0" y="0" width="300" height="300"><rect x="0" y="0" width="300" height="300" fill="#ffffff" />'
for _segment_ in my_segments:
    svg6 += f'<line x1="{_segment_[0][0]}" y1="{_segment_[0][1]}" x2="{_segment_[1][0]}" y2="{_segment_[1][1]}" stroke="#000000" stroke-width="2.0"/>'
for _pt_ in my_pts:
    svg6 += f'<line x1="{_pt_[0]-5}" y1="{_pt_[1]-5}" x2="{_pt_[0]+5}" y2="{_pt_[1]+5}" stroke="#000000" stroke-width="2.0"/>'
    svg6 += f'<line x1="{_pt_[0]-5}" y1="{_pt_[1]+5}" x2="{_pt_[0]+5}" y2="{_pt_[1]-5}" stroke="#000000" stroke-width="2.0"/>'
    for _segment_ in my_segments:
        _d_, _p_ = rt.closestPointOnSegment(_segment_, _pt_)
        x, y = _p_[0], _p_[1]
        svg6 += f'<line x1="{_pt_[0]}" y1="{_pt_[1]}" x2="{x}" y2="{y}" stroke="#ff0000" stroke-width="0.5"/>'
svg6 += '</svg>'

svg7 = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
svg7 += f'<path d="M 300 200 C 320 220 420 220 460 100"  stroke="#ff0000" stroke-width="1.2" fill="none" stroke-opacity="0.8" />'
bc = rt.bezierCurve((300,200), (320, 220), (420, 220), (460, 100))
t = 0.0
while t <= 1.0:
    pt = bc(t)
    svg7 += f'<circle cx="{pt[0]}" cy="{pt[1]}" r="10" fill="none" stroke="#000000" stroke-width="0.2" />'
    t += 0.1
svg7 += '</svg>'
rt.tile([svg6,svg7])

In [ ]:
_n_circles_       = 50
_radius_min_      = 10
_radius_max_      = 50
_circle_geoms_    = []
_min_circle_sep_  = 40
_half_sep_        = _min_circle_sep_/2.0   # Needs to be more than the _radius_inc_test_
_radius_inc_test_ = 4
_radius_start_    = _radius_inc_test_ + 1  # Needs to be more than the _radius_inc_test_ ... less than the _min_circle_sep_
_escape_px_       = 20                     # less than the _min_circle_sep_

def circleOverlaps(cx, cy, r):
    for _geom_ in _circle_geoms_:
        dx, dy = _geom_[0] - cx, _geom_[1] - cy
        d      = sqrt(dx*dx+dy*dy)
        if d < (r + _geom_[2] + _min_circle_sep_): # at least 10 pixels apart...
            return True
    return False

def findOpening():
    attempts  = 0
    cx, cy, r = random.randint(_radius_max_, 600-_radius_max_),     random.randint(_radius_max_, 400-_radius_max_), random.randint(_radius_min_,_radius_max_)
    while circleOverlaps(cx,cy,r) and attempts < 20:
        cx, cy, r = random.randint(_radius_max_, 600-_radius_max_), random.randint(_radius_max_, 400-_radius_max_), random.randint(_radius_min_,_radius_max_)
        attempts += 1
    if attempts == 20:
        return None
    return cx, cy, r

for i in range(_n_circles_):
    to_unpack = findOpening()
    if to_unpack is not None:
        cx, cy, r = to_unpack
        _circle_geoms_.append((cx,cy,r))

# points to connect
_pts_ = []
c0    = random.randint(0, len(_circle_geoms_)-1)
c1    =  random.randint(0, len(_circle_geoms_)-1)
while c1 == c0:
    c1 =  random.randint(0, len(_circle_geoms_)-1)
a0, a1 = random.random() * 2 * pi, random.random() * 2 * pi
cx, cy, r = _circle_geoms_[c0]
_pts_.append((cx+(r+_radius_start_+1)*cos(a0), cy+(r+_radius_start_+1)*sin(a0)))
_pts_.append((cx+(r+_escape_px_)*cos(a0), cy+(r+_escape_px_)*sin(a0)))
cx, cy, r = _circle_geoms_[c1]
_pts_.append((cx+(r+_escape_px_)*cos(a1), cy+(r+_escape_px_)*sin(a1)))
_pts_.append((cx+(r+_radius_start_+1)*cos(a1), cy+(r+_radius_start_+1)*sin(a1)))

_pts2_ = [_pts_[0], _pts_[1]]
c      = random.randint(0,len(_circle_geoms_)-1)
while c == c0:
    c  = random.randint(0,len(_circle_geoms_)-1)
a      = random.random() * 2 * pi
cx, cy, r = _circle_geoms_[c]
_pts2_.append((cx+(r+_escape_px_)     *cos(a), cy+(r+_escape_px_)     *sin(a)))
_pts2_.append((cx+(r+_radius_start_+1)*cos(a), cy+(r+_radius_start_+1)*sin(a)))

_pts3_ = [_pts_[0], _pts_[1]]
c      = random.randint(0,len(_circle_geoms_)-1)
while c == c0:
    c  = random.randint(0,len(_circle_geoms_)-1)
a      = random.random() * 2 * pi
cx, cy, r = _circle_geoms_[c]
_pts3_.append((cx+(r+_escape_px_)     *cos(a), cy+(r+_escape_px_)     *sin(a)))
_pts3_.append((cx+(r+_radius_start_+1)*cos(a), cy+(r+_radius_start_+1)*sin(a)))

_pts4_ = [_pts_[0], _pts_[1]]
c      = random.randint(0,len(_circle_geoms_)-1)
while c == c0:
    c  = random.randint(0,len(_circle_geoms_)-1)
a      = random.random() * 2 * pi
cx, cy, r = _circle_geoms_[c]
_pts4_.append((cx+(r+_escape_px_)     *cos(a), cy+(r+_escape_px_)     *sin(a)))
_pts4_.append((cx+(r+_radius_start_+1)*cos(a), cy+(r+_radius_start_+1)*sin(a)))

_pts5_ = [_pts_[0], _pts_[1]]
c      = random.randint(0,len(_circle_geoms_)-1)
while c == c0:
    c  = random.randint(0,len(_circle_geoms_)-1)
a      = random.random() * 2 * pi
cx, cy, r = _circle_geoms_[c]
_pts5_.append((cx+(r+_escape_px_)     *cos(a), cy+(r+_escape_px_)     *sin(a)))
_pts5_.append((cx+(r+_radius_start_+1)*cos(a), cy+(r+_radius_start_+1)*sin(a)))

svg = '<svg width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
for _geom_ in _circle_geoms_:
    _color_ = '#000000'
    _dist_, _inter_  = rt.segmentIntersectsCircle((_pts_[1],_pts_[2]),_geom_)
    if _dist_ <= _geom_[2]:
        _color_ = '#ff0000'
    svg += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="{_color_}" fill-opacity="0.2" />'

_path_ = f'M {_pts_[0][0]} {_pts_[0][1]}'
for i in range(1, len(_pts_)):
    _path_ += f' L {_pts_[i][0]} {_pts_[i][1]}'
svg += f'<path d="{_path_}" fill="none" stroke="#ff0000" stroke-width="0.2" />'

svg += f'<circle cx="{_pts_[0][0]}"  cy="{_pts_[0][1]}"  r="3" stroke="#000000" fill="#ff0000" />'
svg += f'<circle cx="{_pts_[-1][0]}" cy="{_pts_[-1][1]}" r="3" stroke="#000000" fill="#ff0000" />'

svg += '</svg>'
rt.displaySVG(svg)

In [ ]:
def calculatePathAroundCircles(pts, circle_geoms, radius_inc_test, half_sep):
    def breakSegment(_segment_):
        if rt.segmentLength(_segment_) < 2.0:
            return _segment_
        for _geom_ in circle_geoms:
            _circle_plus_ = (_geom_[0], _geom_[1], _geom_[2]+radius_inc_test)
            _dist_, _inter_  = rt.segmentIntersectsCircle(_segment_,_circle_plus_)
            if _dist_ <= _circle_plus_[2]:
                if _inter_[0] == _geom_[0] and _inter_[1] == _geom_[1]:
                    dx, dy   = _segment_[1][0] - _segment_[0][0], _segment_[1][1] - _segment_[0][1]
                    l        = sqrt(dx*dx+dy*dy)
                    dx,  dy  = dx/l, dy/l
                    pdx, pdy = -dy, dx 
                    return [(_segment_[0][0], _segment_[0][1]), (_geom_[0] + pdx*(_geom_[2]+half_sep), _geom_[1] + pdy*(_geom_[2]+half_sep)), (_segment_[1][0], _segment_[1][1])]
                else:
                    dx, dy = _inter_[0] - _geom_[0], _inter_[1] - _geom_[1]
                    l      = sqrt(dx*dx+dy*dy)
                    dx, dy = dx/l, dy/l
                    return [(_segment_[0][0], _segment_[0][1]), (_geom_[0]  + dx*(_geom_[2]+half_sep), _geom_[1]  + dy*(_geom_[2]+half_sep)), (_segment_[1][0], _segment_[1][1])]
        return _segment_

    last_length = 0
    _segments_  = []
    for _pt_ in pts:
        _segments_.append(_pt_)
    while last_length != len(_segments_):
        last_length    = len(_segments_)
        _new_segments_ = []
        for i in range(len(_segments_)-1):
            _new_ = breakSegment([_segments_[i], _segments_[i+1]])
            if len(_new_) == 3:
                _new_segments_.append(_new_[0])
                _new_segments_.append(_new_[1])
            else:
                _new_segments_.append(_new_[0])
        _new_segments_.append(_new_[-1])
        _segments_ = _new_segments_
        
    return _segments_

def createLinePathFromSegments(segments):
    _path_ = f'M {segments[0][0]} {segments[0][1]}'
    for i in range(1,len(segments)):
        _path_ += f' L {segments[i][0]} {segments[i][1]}'
    return _path_

def createCurvedPathFromSegments(segments, amp = 10.0):
    _path_ =  f'M {segments[0][0]} {segments[0][1]}'
    _path_ += f' L {segments[1][0]} {segments[1][1]}'
    for i in range(1,len(segments)-2):
        v0 = rt.unitVector([segments[i],   segments[i-1]])
        v1 = rt.unitVector([segments[i+1], segments[i+2]])
        _path_ += f' C {segments[i][0]-amp*v0[0]} {segments[i][1]-amp*v0[1]} {segments[i+1][0]-amp*v1[0]} {segments[i+1][1]-amp*v1[1]} {segments[i+1][0]} {segments[i+1][1]}'
    _path_ += f' L {segments[-1][0]} {segments[-1][1]}'
    return _path_


In [ ]:
my_segments  = calculatePathAroundCircles(_pts_,  _circle_geoms_, _radius_inc_test_, _half_sep_)
my_segments2 = calculatePathAroundCircles(_pts2_, _circle_geoms_, _radius_inc_test_, _half_sep_)
my_segments3 = calculatePathAroundCircles(_pts3_, _circle_geoms_, _radius_inc_test_, _half_sep_)
my_segments4 = calculatePathAroundCircles(_pts4_, _circle_geoms_, _radius_inc_test_, _half_sep_)
my_segments5 = calculatePathAroundCircles(_pts5_, _circle_geoms_, _radius_inc_test_, _half_sep_)

svg2 = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
for _geom_ in _circle_geoms_:
    _color_ = '#000000'
    svg2 += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="{_color_}" fill-opacity="0.2" />'
svg2 += f'<path d="{createLinePathFromSegments(my_segments)}" stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg2 += f'<circle cx="{_pts_[0][0]}"  cy="{_pts_[0][1]}"  r="3" stroke="#000000" fill="#ff0000" />'
svg2 += f'<circle cx="{_pts_[-1][0]}" cy="{_pts_[-1][1]}" r="3" stroke="#000000" fill="#ff0000" />'
svg2 += '</svg>'

rt.tile([svg,svg2])

In [ ]:
svg3 = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
for _geom_ in _circle_geoms_:
    _color_ = '#000000'
    svg3 += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="{_color_}" fill-opacity="0.2" />'
svg3 += f'<path d="{createCurvedPathFromSegments(my_segments)}"  stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg3 += f'<circle cx="{_pts_[ 0][0]}"  cy="{_pts_[0][1]}"   r="1" stroke="#000000" fill="#ff0000" />'
svg3 += f'<circle cx="{_pts_[-1][0]}"  cy="{_pts_[-1][1]}"  r="1" stroke="#000000" fill="#ff0000" />'
svg3 += '</svg>'
rt.tile([svg2,svg3])

In [ ]:
svg4l = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
for _geom_ in _circle_geoms_:
    _color_ = '#000000'
    svg4l += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="{_color_}" fill-opacity="0.2" />'
svg4l += f'<path d="{createLinePathFromSegments(my_segments)}"  stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg4l += f'<path d="{createLinePathFromSegments(my_segments2)}" stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg4l += f'<path d="{createLinePathFromSegments(my_segments3)}" stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg4l += f'<path d="{createLinePathFromSegments(my_segments4)}" stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg4l += f'<path d="{createLinePathFromSegments(my_segments5)}" stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg4l += f'<circle cx="{_pts_[ 0][0]}"  cy="{_pts_[0][1]}"   r="3" stroke="#0000ff" fill="#0000ff" />'
svg4l += f'<circle cx="{_pts_[-1][0]}"  cy="{_pts_[-1][1]}"  r="1" stroke="#000000" fill="#ff0000" />'
svg4l += f'<circle cx="{_pts2_[-1][0]}" cy="{_pts2_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg4l += f'<circle cx="{_pts3_[-1][0]}" cy="{_pts3_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg4l += f'<circle cx="{_pts4_[-1][0]}" cy="{_pts4_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg4l += f'<circle cx="{_pts5_[-1][0]}" cy="{_pts5_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg4l += '</svg>'

svg4 = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
for _geom_ in _circle_geoms_:
    _color_ = '#000000'
    svg4 += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="{_color_}" fill-opacity="0.2" />'
svg4 += f'<path d="{createCurvedPathFromSegments(my_segments)}"  stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg4 += f'<path d="{createCurvedPathFromSegments(my_segments2)}" stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg4 += f'<path d="{createCurvedPathFromSegments(my_segments3)}" stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg4 += f'<path d="{createCurvedPathFromSegments(my_segments4)}" stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg4 += f'<path d="{createCurvedPathFromSegments(my_segments5)}" stroke="#ff0000" stroke-width="1.2" fill="none" />'
svg4 += f'<circle cx="{_pts_[ 0][0]}"  cy="{_pts_[0][1]}"   r="3" stroke="#0000ff" fill="#0000ff" />'
svg4 += f'<circle cx="{_pts_[-1][0]}"  cy="{_pts_[-1][1]}"  r="1" stroke="#000000" fill="#ff0000" />'
svg4 += f'<circle cx="{_pts2_[-1][0]}" cy="{_pts2_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg4 += f'<circle cx="{_pts3_[-1][0]}" cy="{_pts3_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg4 += f'<circle cx="{_pts4_[-1][0]}" cy="{_pts4_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg4 += f'<circle cx="{_pts5_[-1][0]}" cy="{_pts5_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg4 += '</svg>'
rt.tile([svg4l,svg4])

In [ ]:
#
# calculateCombinedPathAroundCircles()
# - entry_segment - [(x0,y0), (x1,y1)] - x0,y0 is the end point of the connection
# - exit_segments - [[(x0,y0), (x1,y1)] , ...] - x1, y1 is the end point of the connection
#
def calculateCombinedPathAroundCircles(entry_segment, exit_segments, circle_geoms, radius_inc_test, half_sep):
    x0, y0 = entry_segment[1]
    # Sort the points by furthest...
    _sorter_ = []
    for i in range(len(exit_segments)):
        x1, y1 = exit_segments[i][0]
        dx, dy = x1 - x0, y1 - y0
        _sorter_.append((dx*dx+dy*dy, i)) # don't need to do the square root
    _sorter_ = sorted(_sorter_)
    # with the furthest point, perform the initial connection
    i_furthest = _sorter_[-1][1]
    x1, y1 = exit_segments[i_furthest][1]
    _segments_ = calculatePathAroundCircles([(x1, y1), (x0, y0)], circle_geoms, radius_inc_test, half_sep)
    _segments_.append(entry_segment[0])
    _segments_.insert(1, exit_segments[i_furthest][0])
    _tributes_ = [_segments_]
    _closests_ = []
    # make the connection for all of the exit segments...
    if len(exit_segments) > 1:
        for i in range(len(_sorter_)-1, -1, -1):
            x1, y1 = exit_segments[i][1]
            # find the closest place to insert this new
            closest_d, closest_x, closest_y, closest_segment = None, None, None, None
            for j in range(len(_tributes_)):
                _segments_ = _tributes_[j]
                for k in range(1, len(_segments_)-2):
                    xa,ya,xb,yb = _segments_[k][0], _segments_[k][1], _segments_[k+1][0], _segments_[k+1][1]
                    _d_ , _c_ = rt.closestPointOnSegment(((xa, ya), (xb, yb)), (x1, y1))
                    xc, yc = _c_[0], _c_[1]
                    if closest_d is None:
                        closest_d, closest_x, closest_y, closest_segment = _d_, xc, yc, _segments_[k+1]
                    elif _d_ < closest_d:
                        closest_d, closest_x, closest_y, closest_segment = _d_, xc, yc, _segments_[k+1]
            _closests_.append((closest_x,closest_y))
            _segments_ = calculatePathAroundCircles([(x1,y1),(closest_x,closest_y)], circle_geoms, radius_inc_test, half_sep)
            _segments_.insert(1, exit_segments[i][0])
            # _segments_.append(closest_segment)
            _tributes_.append(_segments_)
    return _tributes_, _closests_

_exit_segments_ = [_pts_[2:] , _pts2_[2:] , _pts3_[2:], _pts4_[2:], _pts5_[2:]]

my_tributes, my_closests = calculateCombinedPathAroundCircles(_pts_[:2], _exit_segments_, _circle_geoms_, _radius_inc_test_, _half_sep_)

svg5 = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
for _geom_ in _circle_geoms_:
    _color_ = '#000000'
    svg5 += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="{_color_}" fill-opacity="0.2" />'
for _segment_ in my_tributes:
    svg5 += f'<path d="{createLinePathFromSegments(_segment_)}"  stroke="#ff0000" stroke-width="1.2" fill="none" />'
for _closest_ in my_closests:
    svg5 += f'<circle cx="{_closest_[0]}" cy="{_closest_[1]}" r="{4}" stroke="#ff0000" fill="none"  />'
svg5 += f'<circle cx="{_pts_[ 0][0]}"  cy="{_pts_[0][1]}"   r="3" stroke="#0000ff" fill="#0000ff" />'
svg5 += f'<circle cx="{_pts_[-1][0]}"  cy="{_pts_[-1][1]}"  r="1" stroke="#000000" fill="#ff0000" />'
svg5 += f'<circle cx="{_pts2_[-1][0]}" cy="{_pts2_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg5 += f'<circle cx="{_pts3_[-1][0]}" cy="{_pts3_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg5 += f'<circle cx="{_pts4_[-1][0]}" cy="{_pts4_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg5 += f'<circle cx="{_pts5_[-1][0]}" cy="{_pts5_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg5 += '</svg>'

rt.tile([svg4, svg5])

In [ ]:
_path_, _all_points_ = '', ''
for i in range(len(my_tributes)):
    _segments_ = my_tributes[i]
    _path_ += ' ' + createCurvedPathFromSegments(_segments_)
    for k in range(len(_segments_)):
        _x_, _y_ = _segments_[k]
        _all_points_ += f'<line x1="{_x_-5}" y1="{_y_-5}" x2="{_x_+5}" y2="{_y_+5}" stroke="#000000" stroke-width="0.5" />'
        _all_points_ += f'<line x1="{_x_+5}" y1="{_y_-5}" x2="{_x_-5}" y2="{_y_+5}" stroke="#000000" stroke-width="0.5" />'

svg6x = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
for _geom_ in _circle_geoms_:
    _color_ = '#000000'
    svg6x += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="{_color_}" fill-opacity="0.2" />'
svg6x += f'<path d="{_path_}"  stroke="#ff0000" stroke-width="1.2" fill="none" stroke-opacity="0.8" />'
svg6x += _all_points_
svg6x += f'<circle cx="{_pts_[ 0][0]}"  cy="{_pts_[0][1]}"   r="3" stroke="#0000ff" fill="#0000ff" />'
svg6x += f'<circle cx="{_pts_[-1][0]}"  cy="{_pts_[-1][1]}"  r="1" stroke="#000000" fill="#ff0000" />'
svg6x += f'<circle cx="{_pts2_[-1][0]}" cy="{_pts2_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg6x += f'<circle cx="{_pts3_[-1][0]}" cy="{_pts3_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg6x += f'<circle cx="{_pts4_[-1][0]}" cy="{_pts4_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg6x += f'<circle cx="{_pts5_[-1][0]}" cy="{_pts5_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg6x += '</svg>'

svg6 = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
for _geom_ in _circle_geoms_:
    _color_ = '#000000'
    svg6 += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="{_color_}" fill-opacity="0.2" />'
svg6 += f'<path d="{_path_}"  stroke="#ff0000" stroke-width="1.2" fill="none" stroke-opacity="0.8" />'
svg6 += f'<circle cx="{_pts_[ 0][0]}"  cy="{_pts_[0][1]}"   r="3" stroke="#0000ff" fill="#0000ff" />'
svg6 += f'<circle cx="{_pts_[-1][0]}"  cy="{_pts_[-1][1]}"  r="1" stroke="#000000" fill="#ff0000" />'
svg6 += f'<circle cx="{_pts2_[-1][0]}" cy="{_pts2_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg6 += f'<circle cx="{_pts3_[-1][0]}" cy="{_pts3_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg6 += f'<circle cx="{_pts4_[-1][0]}" cy="{_pts4_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg6 += f'<circle cx="{_pts5_[-1][0]}" cy="{_pts5_[-1][1]}" r="1" stroke="#000000" fill="#ff0000" />'
svg6 += '</svg>'

rt.tile([svg6x, svg6])

In [ ]:
all_paths = [my_segments, my_segments2, my_segments3, my_segments4, my_segments5]
entries   = [_pts_,       _pts2_,       _pts3_,       _pts4_,       _pts5_]

svg8 = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
# Render Cirlces
for _geom_ in _circle_geoms_:
    svg8 += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="#000000" fill-opacity="0.2" />'
# Render Paths
for _path_ in all_paths:
    for i in range(len(_path_)-1):
        svg8 += f'<line x1="{_path_[i][0]}" y1="{_path_[i][1]}" x2="{_path_[i+1][0]}" y2="{_path_[i+1][1]}" stroke="#000000" stroke-width="0.2" />'
# Render Entry Points
svg8 += f'<line   x1="{entries[0][0][0]}" y1="{entries[0][0][1]}" x2="{entries[0][1][0]}" y2="{entries[0][1][1]}" stroke="#00af00" stroke-width="2" />'
svg8 += f'<circle cx="{entries[0][0][0]}" cy="{entries[0][0][1]}" r="3" stroke="#00af00" fill="#00af00" />'
for _entry_ in entries:
    svg8 += f'<line   x1="{_entry_[2][0]}" y1="{_entry_[2][1]}" x2="{_entry_[3][0]}" y2="{_entry_[3][1]}" stroke="#ff0000" stroke-width="2" />'
    svg8 += f'<circle cx="{_entry_[3][0]}" cy="{_entry_[3][1]}" r="3" stroke="#ff0000" fill="#ff0000" />'
svg8 += '</svg>'


def expandSegmentsIntoPiecewiseCurveParts(segments, amp=5.0, ampends=20.0, t_inc=0.1):
    _piecewise_ = [segments[0], segments[1]]
    for i in range(1,len(segments)-2):
        _amp_ = ampends if ((i == 1) or (i == len(segments)-3)) else amp
        v0 = rt.unitVector([segments[i],   segments[i-1]])
        v1 = rt.unitVector([segments[i+1], segments[i+2]])
        bc = rt.bezierCurve(segments[i], ( segments[i][0]-_amp_*v0[0] , segments[i][1]-_amp_*v0[1] ), ( segments[i+1][0]-_amp_*v1[0] , segments[i+1][1]-_amp_*v1[1] ), segments[i+1])
        t = 0.0
        while t < 1.0:
            _piecewise_.append(bc(t))
            t += t_inc
    _piecewise_.append(segments[-1])
    return _piecewise_

def smoothSegments(segments):
    smoothed = [segments[0]]
    for i in range(1, len(segments)-1):
        x, y = (segments[i-1][0] + segments[i][0] + segments[i+1][0])/3.0 , (segments[i-1][1] + segments[i][1] + segments[i+1][1])/3.0
        smoothed.append((x,y))
    smoothed.append(segments[-1])
    return smoothed

t0 = time.time()
svg8pw = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
# Render Cirlces
for _geom_ in _circle_geoms_:
    svg8pw += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#000000" fill="#000000" fill-opacity="0.2" />'
# Render Paths
for _path_ in all_paths:
    _piecewise_ = smoothSegments(smoothSegments(expandSegmentsIntoPiecewiseCurveParts(_path_)))
    for i in range(len(_piecewise_)-1):
        _x1_, _y1_ = _piecewise_[i]
        _x2_, _y2_ = _piecewise_[i+1]
        svg8pw += f'<line x1="{_x1_}" y1="{_y1_}" x2="{_x2_}" y2="{_y2_}" stroke="#000000" stroke-width="0.2" />'
        #svg8pw += f'<line x1="{_x1_-2}" y1="{_y1_-2}" x2="{_x1_+2}" y2="{_y1_+2}" stroke="#000000" stroke-width="0.4" />'
        #svg8pw += f'<line x1="{_x1_+2}" y1="{_y1_-2}" x2="{_x1_-2}" y2="{_y1_+2}" stroke="#000000" stroke-width="0.4" />'
# Render Entry Points
svg8pw += f'<line   x1="{entries[0][0][0]}" y1="{entries[0][0][1]}" x2="{entries[0][1][0]}" y2="{entries[0][1][1]}" stroke="#00af00" stroke-width="2" />'
svg8pw += f'<circle cx="{entries[0][0][0]}" cy="{entries[0][0][1]}" r="3" stroke="#00af00" fill="#00af00" />'
for _entry_ in entries:
    svg8pw += f'<line   x1="{_entry_[2][0]}" y1="{_entry_[2][1]}" x2="{_entry_[3][0]}" y2="{_entry_[3][1]}" stroke="#ff0000" stroke-width="2" stroke-opacity="0.2" />'
    svg8pw += f'<circle cx="{_entry_[3][0]}" cy="{_entry_[3][1]}" r="3" stroke="#ff0000" fill="#ff0000" />'
svg8pw += '</svg>'
t1 = time.time()

print(t1-t0)
rt.tile([svg8, svg8pw])

In [ ]:
class SegmentOctTree(object):
    #    
    # bounds == (x0,y0,x1,y1)
    #
    def __init__(self, bounds, max_segments_per_cell=20):
        self.bounds                 = bounds
        self.max_segments_per_cell  = max_segments_per_cell
        self.tree                   = {}
        self.tree_bounds            = {}
        self.tree['']               = set()
        self.tree_bounds['']        = self.bounds
        self.tree_already_split     = {}
        self.tree_already_split[''] = False

        # For Debugging...
        self.pieces                 = set()            # for debugging...
        debug = False
        if debug:
            self.__split__('')
            iters = 0
            while (iters < 4):
                ks = set(self.tree.keys())
                for k in ks:
                    self.__split__(k)
                iters += 1

    #
    # findOctet() - find octet for point.
    #
    def findOctet(self, pt):
        last_s = s = ''
        b = self.bounds
        while s in self.tree.keys():
            b = self.tree_bounds[s]
            if    pt[0] <= (b[0]+b[2])/2.0 and pt[1] <= (b[1]+b[3])/2.0:
                n = '0'
            elif  pt[0] >  (b[0]+b[2])/2.0 and pt[1] <= (b[1]+b[3])/2.0:
                n = '1'
            elif  pt[0] <= (b[0]+b[2])/2.0 and pt[1] >  (b[1]+b[3])/2.0:
                n = '2'
            elif  pt[0] >  (b[0]+b[2])/2.0 and pt[1] >  (b[1]+b[3])/2.0:
                n = '3'
            last_s = s
            s += n
        return last_s

    #
    # __split__() - split a tree node into four parts ... not thread safe
    #
    def __split__(self, node):
        if self.tree_already_split[node]:
            return
        else:
            self.tree_already_split[node] = True
        
        b = self.tree_bounds[node]
        self.tree       [node+'0'] = set()
        self.tree_bounds[node+'0'] = (b[0],            b[1],            (b[0]+b[2])/2.0, (b[1]+b[3])/2.0)
        self.tree_already_split[node+'0'] = False

        self.tree       [node+'1'] = set()
        self.tree_bounds[node+'1'] = ((b[0]+b[2])/2.0, b[1],            b[2],            (b[1]+b[3])/2.0)        
        self.tree_already_split[node+'1'] = False

        self.tree       [node+'2'] = set()
        self.tree_bounds[node+'2'] = (b[0],            (b[1]+b[3])/2.0, (b[0]+b[2])/2.0, b[3])
        self.tree_already_split[node+'2'] = False

        self.tree       [node+'3'] = set()
        self.tree_bounds[node+'3'] = ((b[0]+b[2])/2.0, (b[1]+b[3])/2.0, b[2],            b[3])
        self.tree_already_split[node+'3'] = False


        to_check =      [node+'0', node+'1', node+'2', node+'3']
        for piece in self.tree[node]:
            x_min, y_min, x_max, y_max = min(piece[0][0], piece[1][0]), min(piece[0][1], piece[1][1]), max(piece[0][0], piece[1][0]), max(piece[0][1], piece[1][1])
            oct0, oct1, piece_addition_count = self.findOctet(piece[0]), self.findOctet(piece[1]), 0
            for k in to_check:
                b = self.tree_bounds[k]                                    
                if   x_max < b[0] or x_min > b[2] or y_max < b[1] or y_min > b[3]:
                        pass
                elif oct0 == oct1 and oct0 == k:
                    self.tree[k].add(piece)
                    piece_addition_count +=1
                elif rt.segmentsIntersect(piece, ((b[0],b[1]),(b[0],b[3]))) or \
                     rt.segmentsIntersect(piece, ((b[0],b[1]),(b[2],b[1]))) or \
                     rt.segmentsIntersect(piece, ((b[2],b[3]),(b[0],b[3]))) or \
                     rt.segmentsIntersect(piece, ((b[2],b[3]),(b[2],b[1]))):
                        self.tree[k].add(piece)
                        piece_addition_count += 1
            if piece_addition_count == 0:
                print(f"Error -- No additions for piece {piece} ... node = {node}")
        self.tree[node] = set()
        for k in to_check:
            if len(self.tree[k]) > self.max_segments_per_cell:
                self.__split__(k)

    #
    # addSegments() -- add segments to the tree
    # - segments = [(x0,y0), (x1,y1), (x2,y2), (x3,y3)]
    def addSegments(self, segments):
        for i in range(len(segments)-1):
            piece = ((segments[i][0], segments[i][1]), (segments[i+1][0], segments[i+1][1])) # make sure it's a tuple
            self.pieces.add(piece)
            oct0  = self.findOctet(segments[i])
            x0,y0 = segments[i]
            oct1  = self.findOctet(segments[i+1])
            x1,y1 = segments[i+1]
            x_min,y_min,x_max,y_max = min(x0,x1), min(y0,y1), max(x0,x1), max(y0,y1)
            if oct0 == oct1:
                self.tree[oct0].add(piece)
                if len(self.tree[oct0]) > self.max_segments_per_cell:
                    self.__split__(oct0)
            else:
                to_split = set() # to avoid messing with the keys in this iteration
                self.tree[oct0].add(piece)
                if len(self.tree[oct0]) > self.max_segments_per_cell:
                    to_split.add(oct0)
                self.tree[oct1].add(piece)
                if len(self.tree[oct1]) > self.max_segments_per_cell:
                    to_split.add(oct1)
                for k in self.tree_bounds.keys():
                    b      = self.tree_bounds[k]
                    if k != oct0 and k != oct1:
                        if   x_max < b[0] or x_min > b[2] or y_max < b[1] or y_min > b[3]:
                             pass
                        elif rt.segmentsIntersect(piece, ((b[0],b[1]),(b[0],b[3]))) or \
                             rt.segmentsIntersect(piece, ((b[0],b[1]),(b[2],b[1]))) or \
                             rt.segmentsIntersect(piece, ((b[2],b[3]),(b[0],b[3]))) or \
                             rt.segmentsIntersect(piece, ((b[2],b[3]),(b[2],b[1]))):
                             self.tree[k].add(piece)
                             if len(self.tree[k]) > self.max_segments_per_cell:
                                to_split.add(k)
                for k in to_split:
                    self.__split__(k)

    #
    # _repr_svg_() - return an SVG version of the oct tree
    #
    def _repr_svg_(self):
        w,  h, x_ins, y_ins = 800, 800, 50, 50
        xa, ya, xb, yb      = self.tree_bounds['']
        xT = lambda x: x_ins + w*(x - xa)/(xb-xa)
        yT = lambda y: y_ins + h*(y - ya)/(yb-ya)
        svg =  f'<svg x="0" y="0" width="{w+2*x_ins}" height="{h+2*y_ins}" xmlns="http://www.w3.org/2000/svg">'
        all_segments = set()
        for k in self.tree:
            all_segments = all_segments | self.tree[k]
            b = self.tree_bounds[k]
            _color_ = rt.co_mgr.getColor(k)
            svg += f'<rect x="{xT(b[0])}" y="{yT(b[1])}" width="{xT(b[2])-xT(b[0])}" height="{yT(b[3])-yT(b[1])}" fill="{_color_}" opacity="0.4" stroke="{_color_}" stroke-width="0.5" stroke-opacity="1.0" />'
        for segment in self.pieces:
            svg += f'<line x1="{xT(segment[0][0])}" y1="{yT(segment[0][1])}" x2="{xT(segment[1][0])}" y2="{yT(segment[1][1])}" stroke="#ffffff" stroke-width="4.0" />'
            nx,  ny  = rt.unitVector(segment)
            pnx, pny = -ny, nx
            svg += f'<line x1="{xT(segment[0][0]) + pnx*3}" y1="{yT(segment[0][1]) + pny*3}" x2="{xT(segment[0][0]) - pnx*3}" y2="{yT(segment[0][1]) - pny*3}" stroke="#000000" stroke-width="0.5" />'
            svg += f'<line x1="{xT(segment[1][0]) + pnx*3}" y1="{yT(segment[1][1]) + pny*3}" x2="{xT(segment[1][0]) - pnx*3}" y2="{yT(segment[1][1]) - pny*3}" stroke="#000000" stroke-width="0.5" />'
        for segment in all_segments:
            svg += f'<line x1="{xT(segment[0][0])}" y1="{yT(segment[0][1])}" x2="{xT(segment[1][0])}" y2="{yT(segment[1][1])}" stroke="#ff0000" stroke-width="2.0" />'
        svg +=  '</svg>'
        return svg


# Bounds
x_min = y_min = x_max = y_max = None
for _path_ in all_paths:
    _piecewise_ = smoothSegments(smoothSegments(expandSegmentsIntoPiecewiseCurveParts(_path_)))
    for pt in _piecewise_:
        if x_min is None:
            x_min = x_max = pt[0]
            y_min = y_max = pt[1]
        else:
            x_min, x_max, y_min, y_max = min(x_min, pt[0]), max(x_max, pt[0]), min(y_min, pt[1]), max(y_max, pt[1])

# Fill In Tree
my_octtree = SegmentOctTree((x_min,y_min,x_max,y_max))
for _path_ in all_paths:
    _piecewise_ = smoothSegments(smoothSegments(expandSegmentsIntoPiecewiseCurveParts(_path_)))
    my_octtree.addSegments(_piecewise_)

rt.tile([my_octtree, svg8pw])